# Load and check image and coordinates

In [30]:
import tifffile
import numpy as np
import pandas as pd
import cv2
from scipy.ndimage import center_of_mass, extrema
import napari 
import matplotlib.pyplot as plt
from skimage import measure
import os

In [31]:
#dir
main_dir = "D:/Internship/NTU/simulation/hi_fission/"
area_estimation_dir = main_dir

im_path = area_estimation_dir + "hi_fiss.ome-TYX-T1p0_Y0p25_X0p25-ch0-t0_to_300-im_obj_label_reassigned.ome.tif"
path = area_estimation_dir + "hi_fiss_event.csv"
feature_csv = area_estimation_dir +  "hi_fiss.ome-TYX-T1p0_Y0p25_X0p25-ch0-t0_to_300-features_organelles.csv"

path1_fis = main_dir + "fission_position_after.log"
path1_fus = main_dir+ "fusion_position_after.log"
path_nonZero = main_dir + "nonzero_frames_final.log"

In [32]:
im = tifffile.imread(im_path)
n_frame, n_y, n_x = im.shape
rescaling_y = im.shape[1]/400
rescaling_x = im.shape[2]/400


scalling_x = 8
scalling_y = 4
translation_x = n_x / 2 #200 # 
translation_y = n_y / 2 #200 # 
rescaling_y,rescaling_x, translation_x , translation_y

(2.12, 1.68, 336.0, 424.0)

In [33]:
square_size = 10  # pixels
image_resolution = [n_x, n_y] 
square_vertices = [
    [
        [t, 0, 0],  # Top-left
        [t, 0, image_resolution[0]],  # Top-right
        [t, image_resolution[1], image_resolution[0]],  # Bottom-right
        [t, image_resolution[1], 0]  # Bottom-left
    ]
    for t in range(im.shape[0])
]

In [34]:
square_size = 10  # pixels
box_size = [400, 400] 
edge_scaling = 210

small_square_vertices = [
    [
        [t, translation_y- scalling_y-edge_scaling, translation_x-scalling_x-edge_scaling],  # Top-left
         [t, translation_y- scalling_y-edge_scaling, translation_x-scalling_x +edge_scaling],# Top-right
         [t, translation_y- scalling_y+edge_scaling, translation_x-scalling_x +edge_scaling],
        [t,translation_y- scalling_y+edge_scaling, translation_x-scalling_x -edge_scaling] 
        
    ]
    for t in range(im.shape[0])
]

In [35]:
tiff = "D:/Internship/NTU/simulation/hi_fission/hi_fiss.ome.tif"
im_tif = tifffile.imread(tiff)
viewer = napari.Viewer()
viewer.add_labels(im_tif, name ="labeled image")
edge_scaling = 198

middle = np.array([1, translation_y- scalling_y, translation_x-scalling_x])[np.newaxis, :]  # shape (1, 3)

left_edge = np.array([1, translation_y- scalling_y-edge_scaling, translation_x-scalling_x-edge_scaling])[np.newaxis, :] 
right_edge = np.array([1, translation_y- scalling_y-edge_scaling, translation_x-scalling_x +edge_scaling])[np.newaxis, :] 

for i in range(1, im.shape[0]):
    new_row = np.array([i+1, translation_y-scalling_y, translation_x-scalling_x])[np.newaxis, :]
    new_l_edge =  np.array([i+1, translation_y- scalling_y-edge_scaling, translation_x-scalling_x-edge_scaling])[np.newaxis, :] 
    new_r_edge = np.array([i+1, translation_y- scalling_y-edge_scaling, translation_x-scalling_x +edge_scaling])[np.newaxis, :] 

    middle = np.vstack((middle, new_row))
    left_edge = np.vstack((left_edge, new_l_edge))
    right_edge = np.vstack((right_edge, new_r_edge))

viewer.add_points(middle, face_color='white', size =2, name = "central point")
viewer.add_points(left_edge, face_color='white', size =2, name = "left edge point")
viewer.add_points(right_edge, face_color='white', size =2, name = "right edge point")

viewer.add_shapes(
    square_vertices,
    shape_type="rectangle",
    edge_color="red",
    face_color="transparent",
    edge_width=2,
    name="Frame Boundary Square"
)

viewer.add_shapes(
    small_square_vertices,
    shape_type="rectangle",
    edge_color="blue",
    face_color="transparent",
    edge_width=2,
    name="Frame Boundary Square"
)


'''
viewer.add_points(middle, face_color='white', size =2, name = "central point")
viewer.add_points(fusion_coords, face_color='yellow', size =2, name = "fusion_node")
viewer.add_points(fission_coords, face_color='yellow', size =2, name = "fission_node")

'''

'\nviewer.add_points(middle, face_color=\'white\', size =2, name = "central point")\nviewer.add_points(fusion_coords, face_color=\'yellow\', size =2, name = "fusion_node")\nviewer.add_points(fission_coords, face_color=\'yellow\', size =2, name = "fission_node")\n\n'

# load function and image

In [10]:
# load position log of both fission and fusion and specific frame
import re
def get_coor (path_to_log_file,rescaleX,rescaleY):

    with open(path_to_log_file, "r", encoding='utf-8-sig') as f:
        string = f.read()
        lines = string.split("\n")
        num = 1
        event_num = []
        x_coordinates = []
        y_coordinates = []
        N_frame= [ ]
        for line in lines:

            if len(line ) != 0:
                if (int(num) - 1)% 3  == 0: 
                    event_num.append(num)
                    num+=1
                    frame_ = line.split(",")
                    frame_val = re.findall(r'\d+', frame_[1])
                    N_frame.append((int(frame_val[0])))

                
                else: 
                    y = line[1:-1].split(',')[1]
                    x = line[1:-1].split(',')[0]
                    x_coord = (rescaleY * (-float(y)))+translation_y-scalling_y
                    #x_coords = list(map(int,x_coord))

                    y_coord = (rescaleX * float(x))+ translation_x-scalling_x
                    #y_coords = list(map(int,y_coord))

                    y_coordinates.append(x_coord)
                    x_coordinates.append(y_coord)
                    #note: with center simulation Y -20 X -35
                    # 
                    num+=1
        N_frame = np.array(N_frame)
        frame_for_2_particle = np.repeat(N_frame,2)
        
    return len(event_num) , pd.DataFrame({'frame': frame_for_2_particle, 'y_coordinates': y_coordinates, 'x_coordinates': x_coordinates}) 

In [11]:
# Load tiff file and log position
#im_path = "D:/Internship/NTU/simulation/with_center/simulation1/multi_center_resized.ome.tif"
#im_path = main_dir + "low_stride.ome.tif"
im = tifffile.imread(im_path)
rescaling_y = im.shape[1]/400
rescaling_x = im.shape[2]/400
rescaling_y,rescaling_x, im.shape

(2.12, 1.68, (301, 848, 672))

In [12]:
num_fission,fission_coor = get_coor(path1_fis, 1, 1)
num_fusion,fusion_coor = get_coor(path1_fus, 1, 1)

print(f"num_fusion {num_fusion}")
print(f"num_fission {num_fission}")



num_fusion 64
num_fission 86


# Load fission fusion frome my script

In [13]:
df = pd.read_csv(path)

mask_fusion = df['isFusion'].astype(int) == 1
fission_event = df[~mask_fusion]
fusion_event = df[mask_fusion]

In [14]:
fission_event

,Volume_pre,Volume_post,Distance,Nearest_Label,y_corr,x_corr,Volume_diff,isFusion,Frame,Label
0,6.8750,6.8125,20.082131,4.0,400.627273,309.636364,-0.4375,0.0,1.0,1.0
1,28.3750,21.0000,20.082131,8.0,467.955947,382.407489,-0.4375,0.0,1.0,1.0
2,5.5625,5.4375,32.031006,3.0,405.640449,335.853933,-0.4375,0.0,2.0,1.0
3,6.9375,6.8750,20.350139,4.0,400.549550,309.423423,-0.4375,0.0,2.0,1.0
4,28.4375,28.3750,20.350139,8.0,468.065934,382.487912,-0.4375,0.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...
2711,0.0000,0.0000,0.000000,0.0,462.647059,387.235294,-0.6250,0.0,293.0,8.0
2712,0.0000,0.0000,0.000000,0.0,474.516129,394.741935,-0.6250,0.0,293.0,8.0
2713,0.0000,0.0000,0.000000,0.0,490.069307,396.841584,-0.6250,0.0,293.0,8.0
2714,0.0000,0.0000,0.000000,0.0,502.160714,395.571429,-0.6250,0.0,293.0,8.0


In [15]:
fusion_event

,Volume_pre,Volume_post,Distance,Nearest_Label,y_corr,x_corr,Volume_diff,isFusion,Frame,Label
7,4.9375,5.1875,2.150063,3.0,405.277108,335.843373,1.1875,1.0,4.0,1.0
14,2.5625,2.6875,30.943338,1.0,381.604651,397.186047,0.3750,1.0,18.0,1.0
15,2.6250,12.4375,3.480102,4.0,402.819095,321.437186,0.3750,1.0,18.0,1.0
16,4.1250,12.4375,3.480102,4.0,402.819095,321.437186,0.3750,1.0,18.0,1.0
30,0.0000,0.0000,0.000000,0.0,355.761905,351.595238,1.3125,1.0,145.0,1.0
...,...,...,...,...,...,...,...,...,...,...
2720,0.0000,0.0000,0.000000,0.0,490.078431,396.892157,0.3125,1.0,295.0,8.0
2721,0.0000,0.0000,0.000000,0.0,502.109091,395.654545,0.3125,1.0,295.0,8.0
2722,0.0000,0.0000,0.000000,0.0,462.194444,386.750000,0.3750,1.0,296.0,8.0
2723,0.0000,0.0000,0.000000,0.0,485.943262,396.446809,0.3750,1.0,296.0,8.0


In [16]:
df_fission = pd.DataFrame({
    'frame': fission_event['Frame'],
    'x_coordinates': fission_event['x_corr'],
    'y_coordinates': fission_event['y_corr'],
})

df_fusion = pd.DataFrame({
    'frame': fusion_event['Frame'],
    'x_coordinates': fusion_event['x_corr'],
    'y_coordinates': fusion_event['y_corr'],
})

In [17]:
df[df['Frame'] == 9]

,Volume_pre,Volume_post,Distance,Nearest_Label,y_corr,x_corr,Volume_diff,isFusion,Frame,Label
1026,0.0,0.0,0.0,0.0,414.990099,305.524752,0.3125,1.0,9.0,5.0
1027,0.0,0.0,0.0,0.0,424.650538,321.543011,0.3125,1.0,9.0,5.0
2292,0.0,0.0,0.0,0.0,478.646617,391.285714,-3.1875,0.0,9.0,8.0
2293,0.0,0.0,0.0,0.0,498.327434,399.486726,-3.1875,0.0,9.0,8.0
2294,0.0,0.0,0.0,0.0,478.646617,391.285714,6.3750,1.0,9.0,8.0
2295,0.0,0.0,0.0,0.0,498.327434,399.486726,6.3750,1.0,9.0,8.0


In [18]:
fis_arr = df_fission.to_numpy()
fis_arr = fis_arr[:,[0,2,1]] # frame, y_coordinates, x_coordinates
fus_arr= df_fusion.to_numpy()# frame, y_coordinates, x_coordinates
fus_arr = fus_arr[:,[0,2,1]] # frame, y_coordinates, x_coordinates
list_fis = []
list_fis = [np.array(fis_list) for fis_list in fis_arr]
list_fus = []
list_fus = [np.array(fus_list) for fus_list in fus_arr]

In [19]:
list_fus

[array([  4.        , 405.27710843, 335.84337349]),
 array([ 18.        , 381.60465116, 397.18604651]),
 array([ 18.        , 402.81909548, 321.43718593]),
 array([ 18.        , 402.81909548, 321.43718593]),
 array([145.        , 355.76190476, 351.5952381 ]),
 array([145.        , 361.57142857, 401.21428571]),
 array([145.        , 364.71212121, 367.8030303 ]),
 array([145.        , 368.71212121, 386.45454545]),
 array([145.        , 374.52525253, 358.46464646]),
 array([145.        , 382.94117647, 397.38235294]),
 array([148.        , 355.71428571, 351.66666667]),
 array([148.     , 361.34375, 401.5    ]),
 array([148.        , 366.55244755, 376.83216783]),
 array([148.        , 374.40206186, 358.7628866 ]),
 array([148.    , 382.9375, 397.5   ]),
 array([151.        , 355.69230769, 351.20512821]),
 array([151.        , 361.59259259, 401.25925926]),
 array([151.        , 365.73148148, 372.9537037 ]),
 array([151.        , 370.5       , 391.26666667]),
 array([151.  , 374.45, 358.52]),

In [20]:
list_fis

[array([  1.        , 400.62727273, 309.63636364]),
 array([  1.        , 467.95594714, 382.40748899]),
 array([  2.        , 405.64044944, 335.85393258]),
 array([  2.        , 400.54954955, 309.42342342]),
 array([  2.        , 468.06593407, 382.48791209]),
 array([  3.        , 368.22865014, 375.0523416 ]),
 array([  3.        , 376.40740741, 353.88888889]),
 array([  6.        , 405.06024096, 335.55421687]),
 array([  7.        , 365.80666667, 371.85666667]),
 array([  7.        , 376.23333333, 354.4       ]),
 array([  7. , 378.5, 358.5]),
 array([  7.        , 382.02564103, 397.69230769]),
 array([ 11.        , 405.81395349, 335.91860465]),
 array([142.   , 355.725, 351.7  ]),
 array([142.        , 361.12121212, 401.39393939]),
 array([142.        , 364.53521127, 367.6056338 ]),
 array([142.     , 366.90625, 382.125  ]),
 array([142.        , 370.48387097, 390.96774194]),
 array([142.        , 374.55670103, 358.21649485]),
 array([142.     , 382.90625, 397.46875]),
 array([144.  

# visualize tiff

In [38]:
tiff = "D:/Internship/NTU/simulation/hi_fission/hi_fiss.ome.tif"
im_tif = tifffile.imread(tiff)
viewer = napari.Viewer()
viewer.add_labels(im_tif, name ="labeled image")
edge_scaling = 198

middle = np.array([1, translation_y- scalling_y, translation_x-scalling_x])[np.newaxis, :]  # shape (1, 3)

left_edge = np.array([1, translation_y- scalling_y-edge_scaling, translation_x-scalling_x-edge_scaling])[np.newaxis, :] 
right_edge = np.array([1, translation_y- scalling_y-edge_scaling, translation_x-scalling_x +edge_scaling])[np.newaxis, :] 

for i in range(1, im.shape[0]):
    new_row = np.array([i+1, translation_y-scalling_y, translation_x-scalling_x])[np.newaxis, :]
    new_l_edge =  np.array([i+1, translation_y- scalling_y-edge_scaling, translation_x-scalling_x-edge_scaling])[np.newaxis, :] 
    new_r_edge = np.array([i+1, translation_y- scalling_y-edge_scaling, translation_x-scalling_x +edge_scaling])[np.newaxis, :] 

    middle = np.vstack((middle, new_row))
    left_edge = np.vstack((left_edge, new_l_edge))
    right_edge = np.vstack((right_edge, new_r_edge))

viewer.add_points(middle, face_color='white', size =2, name = "central point")
viewer.add_points(left_edge, face_color='white', size =2, name = "left edge point")
viewer.add_points(right_edge, face_color='white', size =2, name = "right edge point")
viewer.add_points(np.array(fusion_coor.values.tolist()), face_color='yellow', size =2, name = "fusion_node")
viewer.add_points(np.array(fission_coor.values.tolist()), face_color='yellow', size =2, name = "fission_node")
viewer.add_shapes(
    square_vertices,
    shape_type="rectangle",
    edge_color="red",
    face_color="transparent",
    edge_width=2,
    name="Frame Boundary Square"
)

viewer.add_shapes(
    small_square_vertices,
    shape_type="rectangle",
    edge_color="blue",
    face_color="transparent",
    edge_width=2,
    name="Frame Boundary Square"
)

<Shapes layer 'Frame Boundary Square [1]' at 0x269a65fde70>

In [ ]:
viewer = napari.Viewer()
viewer.add_labels(im_tif, name ="labeled image")
viewer.add_points(middle, face_color='white', size =2, name = "central point")
viewer.add_points(np.array(fusion_coor.values.tolist()), face_color='yellow', size =2, name = "fusion_node")
viewer.add_points(np.array(fission_coor.values.tolist()), face_color='yellow', size =2, name = "fission_node")
viewer.add_shapes(
    square_vertices,
    shape_type="rectangle",
    edge_color="red",
    face_color="transparent",
    edge_width=2,
    name="Frame Boundary Square"
)

<Shapes layer 'Frame Boundary Square' at 0x26983bf4f70>

# run visualization

In [131]:
middle = np.array([1, translation_y- scalling_y, translation_x-scalling_x])[np.newaxis, :]  # shape (1, 3)
for i in range(1, im.shape[0]):
    new_row = np.array([i+1, translation_y-scalling_y, translation_x-scalling_x])[np.newaxis, :]
    middle = np.vstack((middle, new_row))

In [132]:
frames = im.shape[0]
centroids = []
labels= []

nellie_df_2d = pd.read_csv(feature_csv)

for frame in range(frames):
        labeled_im_frame = im[frame]
        unique_label = np.unique(labeled_im_frame)
        unique_label = unique_label[unique_label != 0]
        num_features = len(unique_label) 

        # !!!!!cannot find a label since it always start over if we use range(len(unique_label))
        for label in unique_label:
            all_extrema = center_of_mass(labeled_im_frame == label)
            centroid = all_extrema#[3]
            centroid = np.array(list(centroid))
            centroids.append(np.append(frame, centroid))
            
            nellie_df_2d_label = nellie_df_2d[nellie_df_2d['label'] == label]
            reassigned_label = nellie_df_2d_label['reassigned_label_raw']
            labels.append(int(list(reassigned_label)[0]))
        #add number of assigned label
            #if label == contour_label:
             #       contours = get_label_contour(labeled_im_frame, label)
              #      for contour in contours:
               #         contour_points.append(np.column_stack((np.full(len(contour), frame), contour)))

    

In [133]:
#show image
viewer = napari.Viewer()
viewer.add_labels(im, name ="labeled image")
viewer.add_points(middle, face_color='white', size =2, name = "central point")
viewer.add_points(fusion_coords, face_color='yellow', size =2, name = "fusion_node")
viewer.add_points(fission_coords, face_color='yellow', size =2, name = "fission_node")

viewer.add_points(list_fis, face_color='green', size =2, name = "fission_area")
viewer.add_points(list_fus, face_color='green', size =2, name = "fusion_area")

viewer.add_points(centroids, text = labels, size =0.5, name = "label number")
napari.run()

#TODO is it the correct position of this iamghe

# Back up

In [ ]:
'''
#show image
viewer = napari.Viewer()
viewer.add_labels(im, name ="labeled image")
viewer.add_points(middle, face_color='white', size =5, name = "label number")
viewer.add_points(test_coord, face_color='yellow', size =1 , text = label, name = "label number")
napari.run()
'''


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage

# Create a 2D binary image
img = np.array([[0, 2, 2, 0],
                [0, 2, 2, 0],
                [0, 2, 2, 0],
                [0, 0, 0, 0]])

# Calculate center of mass
com = ndimage.center_of_mass(img)
print(com)
# Plot the image and center of mass
plt.imshow(img, cmap='gray')
plt.scatter(com[1], com[0], c='red', marker='x', s=100)  # Swap row, column for plotting
plt.title('Center of Mass')
plt.show()

In [ ]:
event_all_log = []
with open(path_nonZero, "r", encoding='utf-8-sig') as f:
    string = f.read()
    lines = string.split("\n")
    for line in lines:
        if len(line ) != 0:
            events = line.split(":")[1].split(",")
            frame = line.split(":")[0].split()[1]
           
            if len(events) == 1 :
                continue
            else:
                num_events = [x.split("=")[1] for x in events]
                x= int(num_events[0]) + float(num_events[1])
                num = [float(frame)-2,x,float(num_events[2])]
                event_all_log.append(num)
# frame , fusion, fission 
event_all_log = (np.transpose(np.array(event_all_log)))
print(f"all_fusion {np.sum(event_all_log[1])}")
print(f"all_fission {np.sum(event_all_log[2])}")

In [ ]:
# append frame in the coordinate array. 
# each frame consistingg of at least 2 coordinates, one for each fusion and fission event
fusion_frames = []
fission_frames = []

fusion_idx = 0
fission_idx = 0

for x in range(len(event_all_log[0])):
    if event_all_log[1][x] > 0 :
        frame = event_all_log[0][x]
        r_frame = np.repeat(frame,event_all_log[1][x]*2 )
        if x == 0:
            fusion_frames = r_frame
        else:
            fusion_frames = np.concatenate((fusion_frames,r_frame))
        fusion_idx += 1
    if event_all_log[2][x] > 0:
        frame = event_all_log[0][x]
        r_frame = np.repeat(frame,event_all_log[2][x]*2 )

        if x == 0:
            fission_frames = r_frame
        else:
            fission_frames = np.concatenate((fission_frames,r_frame))

        fission_idx += 1

# Assign the lists as new columns in your DataFrames
fusion_coor['frame'] = fusion_frames
fission_coor['frame'] = fission_frames

In [ ]:
# need 2 loop coz frame are different
fusion_coords = []
for f in range(len(fusion_coor['frame'])):
    frame = int(fusion_coor.iloc[f,2])
    target_frame = im[frame]
    coor = np.append( fusion_coor.iloc[f,0], fusion_coor.iloc[f,1]) # x,y coordinates or yx?
    fusion_coords.append(np.append(frame, coor))

fission_coords = []
for f in range(len(fission_coor['frame'])):
    frame = int(fission_coor.iloc[f,2])
    target_frame = im[frame]
    coor = np.append( fission_coor.iloc[f,0], fission_coor.iloc[f,1]) # x,y coordinates or yx?
    fission_coords.append(np.append(frame, coor))
